In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math

In [3]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
session = tf.Session(config=config)

In [4]:
tf.__version__

'1.11.0'

### 定义神经网络所需要的基本参数

In [5]:
# 第一层卷积层的卷积核大小
filter_size1 = 5
# 第一层卷积核的个数
num_filters1 = 16
# 第二层卷积核的大小
filter_size2 = 5
# 第二层卷积核的大小
num_filter2 = 36
# 全连接层的大小
fc_size = 128

### 载入MNIST 数据集

In [6]:
from mnist import MNIST
data = MNIST(data_dir="data/MNIST/")

In [7]:
# 查看数据集的大小
print("训练集的大小：", data.num_train)
print("验证集的大小:", data.num_val)
print("测试集的大小:", data.num_test)

训练集的大小： 55000
验证集的大小: 5000
测试集的大小: 10000


### 复制数据维度信息

In [32]:
# 图片展开的大小
img_size_flat = data.img_size_flat

#图片原始尺寸
img_shape = data.img_shape

# 类别数
num_classes = data.num_classes

#每个维度上图片的像素个数
img_size = data.img_size

# 图片的通道数
num_channels = data.num_channels

### Tensorflow图简单说明
- 占位符变量(placeholder)用来改变图的输入
- 模型变量(Model)将会被优化，使得模型的表现更好
- 模型本质上就是一些数学函数，它根据placeholder和模型的输入变量来计算一些输出
- 一个cost度量用来指导变量的优化
- 一个优化策略会更新模型的变量

### 创建新变量的帮助函数

In [9]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

In [10]:
tf.truncated_normal??

In [11]:
def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

In [12]:
tf.constant??

### 创建卷积层的帮助函数

假设输入的四维张量，各个维度如下

1.图像数量 

2.每张图像的Y轴 

3.每张图像的X轴 

4.每张图像的通道数 


输入图片的张量的时候可能是彩色的通道即红绿蓝，当输入是前一层卷积层生成的输出的时候，它可能是滤波通道

输出是另外一个四通道的张量，如下
1.图像数量，与输入相同 

2.每张图的Y轴，如果用了2x2的池化，则输入是图像宽高的一半 

3.每张图像的X轴 

4.卷积滤波生成的通道数 


In [17]:
def new_conv_layer(input, num_input_channels, filter_size, num_filters, use_pooling=True):
    # 定义输入张量的形状
    shape = [filter_size, filter_size, num_input_channels, num_filters]
    # 初始化当前层的weights
    weights = new_weights(shape=shape)
    # 初始化biases
    biases = new_biases(length=num_filters)
    # 建立一个卷积
    layer = tf.nn.conv2d(
        input=input,
        filter=weights,
        strides=[1, 1, 1, 1],
        padding='SAME'
    )
    layer += biases
    if use_pooling is True:
        # 进行池化层操作
        # 初始化参数为： 输入层、卷积核大小2x2，步长高宽2，2，填充0
        layer = tf.nn.max_pool(
            value=layer,
            ksize=[1, 2, 2, 1],
            strides=[1, 2, 2, 1],
            padding='SAME'
        )
    # 对卷积的输出或池化层进行激活
    layer = tf.nn.relu(layer)
    return layer, weights
    

In [18]:
tf.reshape??

### 转换一个层的帮助函数

卷积层生成四维的张量，我们会在卷积层后添加一个全连接层，将这个四维张量转换为2维张量

In [19]:
def flatten_layer(layer):
    # 获取layer的shape
    layer_shape = layer.get_shape()
    # layer_shape = [num_images, img_height, img_width, num_channels]
    # 根据维度计算参数个数
    num_features = layer_shape[1:4].num_elements()
    # 展平张量 reshape 其中一个维度设置为-1 表示自动计算剩余的维度
    layer_flat = tf.reshape(layer, shape=[-1, num_features])
    return layer_flat, num_features

### 创建一个全连接层的帮助函数
输入大小是\[num_images, num_inputs\],输出大小是\[num_images, num_outputs\]

In [28]:
def new_fc_layer(input, num_inputs, num_outputs, use_relu=True):
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)
    layer = tf.matmul(input, weights) + biases
    if use_relu is True:
        layer = tf.nn.relu(layer)
    return layer

### 声明占位符变量

In [33]:
# mnist输入图像的大小为多张 形状为img_size_flat大小的向量
x = tf.placeholder(dtype=tf.float32, shape=[None, img_size_flat], name='x')
# 卷积层接受的是[image_num, image_height, image_width, num_channels]的四维张量，需要用reshape进行变换
x_image = tf.reshape(x, shape=[-1, img_size, img_size, num_channels])
# 定义真实类别
y_true = tf.placeholder(dtype=tf.float32, shape=[None, num_classes])
# sparse 真实类别
y_true_cls = tf.argmax(y_true, axis=1)

### 定义第一个卷积层

In [41]:
layer1, weights1 = new_conv_layer(
    input=x_image, 
    num_input_channels=num_channels, 
    filter_size=filter_size1, 
    num_filters=num_filters1, 
    use_pooling=True
)

In [42]:
layer1

<tf.Tensor 'Relu_2:0' shape=(?, 14, 14, 16) dtype=float32>

### 定义第二个卷积层

In [43]:
layer2, weights2 = new_conv_layer(
    input=layer1,
    num_input_channels=num_filters1,
    filter_size=filter_size2,
    num_filters=num_filter2,
    use_pooling=True
)

In [44]:
layer2

<tf.Tensor 'Relu_3:0' shape=(?, 7, 7, 36) dtype=float32>

### 将卷积层展平

In [45]:
layer_flat, num_features = flatten_layer(layer2)

### 建立全连接层1
输出为128维向量

In [48]:
layer_fc1 = new_fc_layer(
    input=layer_flat, 
    num_inputs=num_features, 
    num_outputs=fc_size, 
    use_relu=True
)

### 建立全连接层2
输出为num_classes

In [52]:
layer_fc2 = new_fc_layer(
    input=layer_fc1,
    num_inputs=fc_size,
    num_outputs=num_classes,
    use_relu=False
)

In [53]:
layer_fc1

<tf.Tensor 'Relu_4:0' shape=(?, 128) dtype=float32>

In [54]:
layer_fc2

<tf.Tensor 'add_6:0' shape=(?, 10) dtype=float32>

### 建立损失函数对logits进行损失计算

In [55]:
# 总体的损失估计
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=layer_fc2, labels=y_true)
# 利用平均损失估计进行度量
cost = tf.reduce_mean(cross_entropy)

### 建立优化方法进行优化

In [56]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

### 性能度量

In [58]:
y_pred_cls = tf.argmax(tf.nn.softmax(layer_fc2))
correct_prediction = tf.equal(y_pred_cls, y_true_cls)

In [59]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))

## TODO
1.复习建模的整个流程：初始化变量-》建模-》建立损失函数-》建立优化器-》建立性能评估

2.复习卷积层、池化层、激活函数的参数用法